In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd
import mapclassify as mc

Step0:  
Read Data and get basic county groups

0.0 Report Data

In [2]:
df_counties_original = pd.read_csv("./us-counties.csv", dtype={"date": np.string_, 
                                                             "county": np.string_,
                                                             "state": np.string_,
                                                             "fips": np.string_,
                                                            "cases": np.int32,
                                                            "deaths": np.int32})
#Tansform Unknown fips to '00000' 
df_counties_original['fips'] = df_counties_original['fips'].replace({np.nan: '00000'})
df_counties_original.sort_values("fips", ascending=True)

,date,county,state,fips,cases,deaths
350637,2020-07-19,Unknown,Wisconsin,00000,3509,7
35483,2020-04-05,Unknown,Washington,00000,615,0
209323,2020-06-05,Unknown,Arkansas,00000,244,0
374432,2020-07-27,Unknown,Michigan,00000,13,2
335056,2020-07-15,Unknown,Florida,00000,332,0
...,...,...,...,...,...,...
264778,2020-06-22,St. Thomas,Virgin Islands,78030,39,0
156925,2020-05-18,St. Thomas,Virgin Islands,78030,38,0
184266,2020-05-27,St. Thomas,Virgin Islands,78030,38,0
255392,2020-06-19,St. Thomas,Virgin Islands,78030,39,0


0.1 GeoJSON Data

In [3]:
old_counties_geojson_df = gpd.read_file(r"./counties_update_new.geojson")
old_counties_geojson_df.head(5)

,NAME,state_name,GEOID,population,geometry
0,Bladen,North Carolina,37017,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,37167,61114,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,39153,541810,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,42113,6177,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,48459,40769,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


Group Data to Counties

In [4]:
pivot_counties = pd.pivot_table(df_counties_original , index=['state','county','fips'],
                       columns=['date'])
pivot_counties = pivot_counties.replace(np.nan, 0)
pivot_counties

cases                                              \
date                   2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state   county   fips                                                           
Alabama Autauga  01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin  01003        0.0        0.0        0.0        0.0        0.0   
        Barbour  01005        0.0        0.0        0.0        0.0        0.0   
        Bibb     01007        0.0        0.0        0.0        0.0        0.0   
        Blount   01009        0.0        0.0        0.0        0.0        0.0   
...                           ...        ...        ...        ...        ...   
Wyoming Teton    56039        0.0        0.0        0.0        0.0        0.0   
        Uinta    56041        0.0        0.0        0.0        0.0        0.0   
        Unknown  00000        0.0        0.0        0.0        0.0        0.0   
        Washakie 56043        0.0        0.0        0.0        0.0        0.0   
        Weston   56045        0.0        0.0        0.0        0.0        0.0   

                                                                               \
date                   2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30   
state   county   fips                                                           
Alabama Autauga  01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin  01003        0.0        0.0        0.0        0.0        0.0   
        Barbour  01005        0.0        0.0        0.0        0.0        0.0   
        Bibb     01007        0.0        0.0        0.0        0.0        0.0   
        Blount   01009        0.0        0.0        0.0        0.0        0.0   
...                           ...        ...        ...        ...        ...   
Wyoming Teton    56039        0.0        0.0        0.0        0.0        0.0   
        Uinta    56041        0.0        0.0        0.0        0.0        0.0   
        Unknown  00000        0.0        0.0        0.0        0.0        0.0   
        Washakie 56043        0.0        0.0        0.0        0.0        0.0   
        Weston   56045        0.0        0.0        0.0        0.0        0.0   

                        ...     deaths                                   \
date                    ... 2020-07-18 2020-07-19 2020-07-20 2020-07-21   
state   county   fips   ...                                               
Alabama Autauga  01001  ...       21.0       21.0       21.0       21.0   
        Baldwin  01003  ...       15.0       15.0       15.0       16.0   
        Barbour  01005  ...        3.0        3.0        4.0        4.0   
        Bibb     01007  ...        2.0        2.0        2.0        2.0   
        Blount   01009  ...        1.0        1.0        1.0        1.0   
...                     ...        ...        ...        ...        ...   
Wyoming Teton    56039  ...        1.0        1.0        1.0        1.0   
        Uinta    56041  ...        0.0        0.0        0.0        0.0   
        Unknown  00000  ...        0.0        0.0        0.0        0.0   
        Washakie 56043  ...        5.0        5.0        5.0        5.0   
        Weston   56045  ...        0.0        0.0        0.0        0.0   

                                                                               \
date                   2020-07-22 2020-07-23 2020-07-24 2020-07-25 2020-07-26   
state   county   fips                                                           
Alabama Autauga  01001       21.0       21.0       21.0       21.0       21.0   
        Baldwin  01003       16.0       17.0       18.0       18.0       18.0   
        Barbour  01005        4.0        4.0        4.0        4.0        4.0   
        Bibb     01007        2.0        2.0        2.0        2.0        2.0   
        Blount   01009        1.0        1.0        1.0        1.0        1.0   
...                           ...        ...        ...    

0.3 Get Date Info

In [5]:
date= pivot_counties['cases'].columns
date

Index(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
       '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30',
       ...
       '2020-07-18', '2020-07-19', '2020-07-20', '2020-07-21', '2020-07-22',
       '2020-07-23', '2020-07-24', '2020-07-25', '2020-07-26', '2020-07-27'],
      dtype='object', name='date', length=189)

In [6]:
date_str = np.sort(date)
dt_str_start=np.min(date_str)
dt_str_end=np.max(date_str)

In [7]:
start = datetime.strptime(dt_str_start, "%Y-%m-%d")
end = datetime.strptime(dt_str_end, "%Y-%m-%d")
dt_range = pd.date_range(start=start,end=end)
print(len(dt_range), dt_range)
dt_range_str = list(map(lambda x: x.strftime("%Y-%m-%d"), dt_range.tolist()))
print(len(dt_range_str), dt_range_str)

189 DatetimeIndex(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24',
               '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28',
               '2020-01-29', '2020-01-30',
               ...
               '2020-07-18', '2020-07-19', '2020-07-20', '2020-07-21',
               '2020-07-22', '2020-07-23', '2020-07-24', '2020-07-25',
               '2020-07-26', '2020-07-27'],
              dtype='datetime64[ns]', length=189, freq='D')
189 ['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-

Step1:  
Add Cases Time Series, First Case Date, Death Time Series, First Death Date

In [8]:
# import json
# pivot_counties['cases_ts'] = json.dumps({"values": pivot_counties['cases'].values.tolist()[0]})
# pivot_counties['deaths_ts'] =  json.dumps({"values": pivot_counties['deaths'].values.tolist()[0]})

pivot_counties['cases_ts'] =  pivot_counties['cases'].values.tolist()
pivot_counties['deaths_ts'] =  pivot_counties['deaths'].values.tolist()

Form Dynamic classes for new_case and new_death 

In [9]:
new_case_dict = {}
for i in range(1,len(date)):
    q6 = mc.Quantiles(pivot_counties['cases_ts'].apply(lambda x: x[i] - x[i - 1]).values, k=6)
    new_case_dict[i] = ','.join(q6.bins.astype(int).astype(str))

C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 1
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:890: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 2
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 3
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaco

In [10]:
dynamic_cl_dict = {}
dynamic_cl_dict['county'] = {'new_case':new_case_dict, 'new_death':{}}
dynamic_cl_dict['state'] = {}

In [11]:
import json
with open('dynamic_classes.json','w') as json_file:
    json.dump(dynamic_cl_dict, json_file)

In [12]:
pivot_counties['dt_first_case'] = (pivot_counties['cases'] > 0).idxmax(axis=1)
pivot_counties['dt_first_death'] = (pivot_counties['deaths'] > 0).idxmax(axis=1)
#For death reports, deal with no deaths counties
pivot_counties.loc[pivot_counties['deaths'].iloc[:, -1] <= 0, 'dt_first_death'] = np.nan
pivot_counties.head(5)

cases                                              \
date                  2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state   county  fips                                                           
Alabama Autauga 01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin 01003        0.0        0.0        0.0        0.0        0.0   
        Barbour 01005        0.0        0.0        0.0        0.0        0.0   
        Bibb    01007        0.0        0.0        0.0        0.0        0.0   
        Blount  01009        0.0        0.0        0.0        0.0        0.0   

                                                                              \
date                  2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30   
state   county  fips                                                           
Alabama Autauga 01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin 01003        0.0        0.0        0.0        0.0        0.0   
        Barbour 01005        0.0        0.0        0.0        0.0        0.0   
        Bibb    01007        0.0        0.0        0.0        0.0        0.0   
        Blount  01009        0.0        0.0        0.0        0.0        0.0   

                       ...     deaths                                   \
date                   ... 2020-07-22 2020-07-23 2020-07-24 2020-07-25   
state   county  fips   ...                                               
Alabama Autauga 01001  ...       21.0       21.0       21.0       21.0   
        Baldwin 01003  ...       16.0       17.0       18.0       18.0   
        Barbour 01005  ...        4.0        4.0        4.0        4.0   
        Bibb    01007  ...        2.0        2.0        2.0        2.0   
        Blount  01009  ...        1.0        1.0        1.0        1.0   

                                             \
date                  2020-07-26 2020-07-27   
state   county  fips                          
Alabama Autauga 01001       21.0       21.0   
        Baldwin 01003       18.0       18.0   
        Barbour 01005        4.0        4.0   
        Bibb    01007        2.0        2.0   
        Blount  01009        1.0        1.0   

                                                                cases_ts  \
date                                                                       
state   county  fips                                                       
Alabama Autauga 01001  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Baldwin 01003  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Barbour 01005  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Bibb    01007  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Blount  01009  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                               deaths_ts  \
date                                                                       
state   county  fips                                                       
Alabama Autauga 01001  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Baldwin 01003  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Barbour 01005  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Bibb    01007  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Blount  01009  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                      dt_first_case dt_first_death  
date                                                
state   county  fips                                
Alabama Autauga 01001    2020-03-24     2020-04-06  
        Baldwin 01003    2020-03-14     2020-04-01  
        Barbour 01005    2020-04-03     2020-04-29  
        Bibb    01007    2020-03-30     2020-05-08  
        Blount  01009    2020-03-25     2020-05-17  

[5 rows x 382 columns]

Step2:  
Add today_case, today_new_case, today_death, today_new_death

In [13]:
yesterday = date.values[-2]
pivot_counties['today_case'] = pivot_counties['cases'][dt_str_end]
pivot_counties['today_new_case'] = pivot_counties['cases'][dt_str_end] - pivot_counties['cases'][yesterday]
pivot_counties['today_death'] = pivot_counties['deaths'][dt_str_end]
pivot_counties['today_new_death'] = pivot_counties['deaths'][dt_str_end] - pivot_counties['deaths'][yesterday]

Step3:  
Extract necessary columns

In [14]:
report_df = pivot_counties[['cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']]
report_df = report_df.reset_index()
report_df.columns = ['state','county','fips','cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']
report_df.head(5)

,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Alabama,Autauga,01001,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-24,2020-04-06,965.0,23.0,21.0,0.0
1,Alabama,Baldwin,01003,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-14,2020-04-01,2770.0,62.0,18.0,0.0
2,Alabama,Barbour,01005,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-04-03,2020-04-29,569.0,7.0,4.0,0.0
3,Alabama,Bibb,01007,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-30,2020-05-08,334.0,10.0,2.0,0.0
4,Alabama,Blount,01009,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-25,2020-05-17,669.0,23.0,1.0,0.0


In [15]:
report_df['cases_ts'][1]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 3.0,
 4.0,
 4.0,
 5.0,
 5.0,
 10.0,
 15.0,
 18.0,
 19.0,
 23.0,
 25.0,
 28.0,
 29.0,
 34.0,
 38.0,
 42.0,
 49.0,
 59.0,
 59.0,
 66.0,
 71.0,
 78.0,
 87.0,
 98.0,
 102.0,
 103.0,
 109.0,
 114.0,
 117.0,
 123.0,
 132.0,
 143.0,
 147.0,
 154.0,
 161.0,
 168.0,
 171.0,
 173.0,
 174.0,
 175.0,
 181.0,
 187.0,
 188.0,
 189.0,
 196.0,
 205.0,
 208.0,
 216.0,
 222.0,
 224.0,
 227.0,
 231.0,
 243.0,
 244.0,
 254.0,
 254.0,
 260.0,
 262.0,
 270.0,
 270.0,
 271.0,
 273.0,
 274.0,
 274.0,
 277.0,
 282.0,
 281.0,
 282.0,
 283.0,
 290.0,
 306.0,
 308.0,
 308.0,
 309.0,
 312.0,
 320.0,
 329.0,
 335.0,
 340.0,
 346.0,
 358.0,
 368.0,
 

Step4:  
Assign Geometries to counties

In [16]:
final_df = pd.merge(old_counties_geojson_df[["NAME", "state_name", "GEOID", "population", "geometry"]], report_df, how='left', left_on=['NAME','state_name'], right_on = ['county','state'])
final_df.columns

Index(['NAME', 'state_name', 'GEOID', 'population', 'geometry', 'state',
       'county', 'fips', 'cases_ts', 'deaths_ts', 'dt_first_case',
       'dt_first_death', 'today_case', 'today_new_case', 'today_death',
       'today_new_death'],
      dtype='object')

Step5:  
Deal with nan values (Counties not shows up in geojson) (More works)

In [17]:
final_df[['today_case','today_new_case','today_death','today_new_death']] = final_df[['today_case','today_new_case','today_death','today_new_death']].replace(np.nan,0)
final_df['fips'] = final_df['fips'].replace(np.nan, final_df['GEOID'])


In [18]:
template =final_df[~final_df['cases_ts'].isna()]['cases_ts']
template = template.iloc[0]

In [19]:
for x in range(0,len(template)):
    template[x] = 0
template = ','.join(map(str, template))

In [20]:

final_df['cases_ts'] = final_df['cases_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else template)
final_df['deaths_ts'] = final_df['deaths_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else template)
final_df.head(5)

,NAME,state_name,GEOID,population,geometry,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Bladen,North Carolina,37017,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",North Carolina,Bladen,37017,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-03,2020-05-06,536.0,8.0,5.0,0.0
1,Stanly,North Carolina,37167,61114,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ...",North Carolina,Stanly,37167,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-20,2020-04-09,793.0,15.0,6.0,0.0
2,Summit,Ohio,39153,541810,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ...",Ohio,Summit,39153,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-14,2020-03-27,2951.0,37.0,214.0,2.0
3,Sullivan,Pennsylvania,42113,6177,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ...",Pennsylvania,Sullivan,42113,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-04,NaN,10.0,0.0,0.0,0.0
4,Upshur,Texas,48459,40769,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ...",Texas,Upshur,48459,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-20,2020-07-25,164.0,3.0,2.0,1.0


Step6:  
Finalize the Dataframe

In [21]:
final_df=final_df[["NAME", "state_name", "population", "fips", "dt_first_case", "dt_first_death", "cases_ts", "deaths_ts", 
                   'today_case','today_new_case','today_death','today_new_death', "geometry"]]
final_df['start'] = dt_str_start
final_df['end'] = dt_str_end
final_df['dt_unit'] = "day"
final_df.head(1)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit
0,Bladen,North Carolina,33778,37017,2020-04-03,2020-05-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",536.0,8.0,5.0,0.0,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",2020-01-21,2020-07-27,day


In [22]:
final_df['change_ts'] = ""
for x in range(0, len(final_df)):
    newCaseList = [0]
    changeRateList = []
    caseStrList = final_df.loc[x,'cases_ts'].split(",")
    for i in range(1, len(caseStrList)):
        dailyNewCase = float(caseStrList[i]) - float(caseStrList[i-1])
        newCaseList.append(dailyNewCase)
    for i in range(0, len(newCaseList)):
        if i < 13:
            changeRate = 0
        else:
            currentWeekSum = 0
            previousWeekSum = 0
            for j in range(0,7):
                currentWeekSum = currentWeekSum + newCaseList[i-j]
            for k in range(7,14):
                previousWeekSum = previousWeekSum + newCaseList[i-k]
            if previousWeekSum == 0 and currentWeekSum == 0:
                changeRate = 0
            elif previousWeekSum == 0 and currentWeekSum != 0:
                changeRate = currentWeekSum
            else:
                changeRate = round(currentWeekSum/previousWeekSum-1,2)
        changeRateList.append(changeRate)
        changeRateStrList = list(map(str, changeRateList))
#     print(','.join(changeRateStrList))
    final_df.loc[x,'change_ts'] = ','.join(changeRateStrList)

final_df

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit,change_ts
0,Bladen,North Carolina,33778,37017,2020-04-03,2020-05-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",536.0,8.0,5.0,0.0,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
1,Stanly,North Carolina,61114,37167,2020-03-20,2020-04-09,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",793.0,15.0,6.0,0.0,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2951.0,37.0,214.0,2.0,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",10.0,0.0,0.0,0.0,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
4,Upshur,Texas,40769,48459,2020-03-20,2020-07-25,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",164.0,3.0,2.0,1.0,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3212,Seward,Nebraska,17127,31159,2020-04-05,2020-04-21,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",66.0,1.0,1.0,0.0,"MULTIPOLYGON (((-97.36812 41.04695, -96.91094 ...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3213,Burke,North Carolina,89712,37023,2020-03-25,2020-04-05,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1439.0,15.0,24.0,0.0,"MULTIPOLYGON (((-81.90665 35.88338, -81.94319 ...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3214,Sumter,Georgia,30352,13261,2020-03-20,2020-03-28,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",698.0,5.0,55.0,0.0,"MULTIPOLYGON (((-84.43301 32.04196, -84.43121 ...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3215,Kansas City,Missouri,491918,00000,2020-03-20,2020-04-02,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",4949.0,164.0,56.0,2.0,"POLYGON ((-94.44849 38.93168, -94.44869 38.928...",2020-01-21,2020-07-27,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."


Step 7:  
Output file

In [23]:
final_df.to_file(r"./nyt_counties_data.geojson", driver='GeoJSON', encoding='utf-8')
print("done")

done


In [24]:
# illinois_gdf = final_df[final_df['state_name'] == 'Illinois']

In [25]:
# illinois_gdf.to_file(r"./illinois/nyt_illinois_counties_data.geojson", driver='GeoJSON', encoding='utf-8')
# print('done')

New Data Structure (Full version)

In [26]:
# pivot_counties.columns = ['_'.join(col).strip() for col in pivot_counties.columns.values]
# report_df_2 = pivot_counties.reset_index()

In [27]:
# report_df_2.head(5)

In [28]:
# pivot_counties.head(5)

In [29]:
# final_df_2 = pd.merge(old_counties_geojson_df[["NAME", "state_name", "population", "geometry"]], report_df_2, how='right', left_on=['NAME','state_name'], right_on = ['county','state'])
# final_df_2 = final_df_2.drop(['state','county'], axis = 1)
# final_df_2['start'] = start
# final_df_2['end'] = end
# final_df_2.head(5)

In [30]:
# final_df_2.to_file(r"./nyt_counties_data_full.geojson", driver='GeoJSON', encoding='utf-8')
# print("done")

Generate JSON file

In [31]:
# county_list = []
# for state in df_counties_original.state.unique():
#     for county in df_counties_original.loc[(df_counties_original['state'] == state)].county.unique():
#         county_list.append({"name": county, "state": state})
 
# print(len(county_list), county_list)


In [32]:
# data_df = report_df.set_index(report_df["county"]+", "+report_df["state"])
# data_dict =data_df.to_dict(orient='index')

In [33]:
# df = df_counties_original.groupby(['county', 'state'], sort=False)[['cases', 'deaths']].max().sort_values("cases", ascending=False)
# county_state_list = list(map(lambda x: x[0]+", "+x[1], df.index.tolist()))
# meta = {"dates": dt_range_str,"states": county_state_list, "cases": df["cases"].values.tolist(), "deaths": df["deaths"].values.tolist()}
# data_dict["metadata"] = meta

In [34]:
# data_dict

In [35]:
# import json
# with open('./nyt_counties_data.json', 'w') as outfile:
#     json.dump(data_dict, outfile)